In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
!pip uninstall wandb -y

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-6erk_in4/unsloth_239f2fa51a3d46abbc4cff692126e79d
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-6erk_in4/unsloth_239f2fa51a3d46abbc4cff692126e79d
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
print(torch.__version__)
from unsloth import FastLanguageModel
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

2.5.1+cu121
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


(8, 9)

In [4]:
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [5]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/Qwen2-0.5b-bnb-4bit",
]
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-0.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.10: Fast Qwen2 patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.10 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs       = examples["input"]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [15]:
train_dataset = load_dataset("prismdata/KDI-DATASET-2014", split="train")
train_dataset = train_dataset.map(formatting_prompts_func, batched=True, )

valid_dataset = load_dataset("prismdata/KDI-DATASET-2014", split="valid")
valid_dataset = valid_dataset.map(formatting_prompts_func, batched=True, )

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=10,
        max_steps=300,
        do_eval=True,
        evaluation_strategy="steps",
        logging_steps=1,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
    ),
)
trainer_stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 304 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 300
 "-____-"     Number of trainable parameters = 8,798,208


Step,Training Loss,Validation Loss
1,0.053600,2.047955
2,0.112300,2.051342
3,0.082900,2.058483
4,0.110400,2.069386
5,0.068800,2.082872
6,0.147600,2.106206
7,0.058800,2.127325
8,0.126900,2.138299
9,0.182500,2.162561
10,0.106000,2.173271


In [16]:
model.save_pretrained("KDI-Qwen2-prismdata")

In [17]:
base_model = "unsloth/Qwen2-0.5B"  # 병합을 수행할 베이스 모델
save_method = ("merged_16bit")

model.save_pretrained_merged(
    base_model,
    tokenizer,
    save_method=save_method,
)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 457.3M


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.68 out of 52.96 RAM for saving.


100%|██████████| 24/24 [00:00<00:00, 99.90it/s] 


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [19]:
# Hub 에 업로드
huggingface_repo = ''
huggingface_token = ''
model.push_to_hub_merged(
    huggingface_repo,
    tokenizer,
    save_method=save_method,
    token=huggingface_token,
)

Unsloth: You are pushing to hub, but you passed your HF username = prismdata.
We shall truncate prismdata/KDI-Qwen2-instruction-0.5B to KDI-Qwen2-instruction-0.5B


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.51 out of 52.96 RAM for saving.


100%|██████████| 24/24 [00:00<00:00, 176.37it/s]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

...

 Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


README.md:   0%|          | 0.00/576 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/prismdata/KDI-Qwen2-instruction-0.5B
